In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
from abcpy.core import *
from abcpy.distributions import *
from abcpy.methods import Rejection

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
def mean(y):
    mu = np.mean(y, axis=1, keepdims=True)
    return mu

def distance(x, y):
    d = np.linalg.norm( np.array(x) - np.array(y), ord=2, axis=0)
    return d

In [ ]:
n = 1000
mu = 1.6

# Set up some observed data y
y = mu + np.random.randn(n)

# Plot
plt.hist(y);

In [ ]:
# Specify the graphical model
mu = Prior('mu', 'uniform', 0, 4)
Y = Model('Y', 'norm', mu, observed=y)
S1 = Summary('S1', mean, Y)
d = Discrepancy('d', distance, S1)

# Specify the number of simulations and set up rejection sampling
N = 10000
rej = Rejection(N, d, [mu], 10000)

In [ ]:
# Time and run parallel
%time result = rej.infer(quantile=0.1)
mu_post = result['samples'][0]

print("Samples: {} ({:.2f}%)".format(len(mu_post), len(mu_post)/N*100))
print("Threshold: {:.2f}".format(result['threshold']))
print("Posterior mean: {:.2f}".format(mu_post.mean()))

In [ ]:
# Changing the threshold doesn't require further sampling.
%time result = rej.infer(threshold=0.01)
mu_post2 = result['samples'][0]

print("Samples: {} ({:.2f}%)".format(len(mu_post2), len(mu_post2)/N*100))
print("Threshold: {:.2f}".format(result['threshold']))
print("Posterior mean: {:.2f}".format(mu_post2.mean()))

In [ ]:
if len(mu_post) > 0:
    plt.hist(mu_post, bins=20)
    plt.title("Posterior for $\mu$")
else:
    print("No accepted samples")